# OCI Forecasting Service Workshop

In [1]:
# Download sample data

## get primary data
!wget https://oracle.github.io/learning-library/oci-library/oci-hol/oci-artificial-intelligence/forecasting-service/item-4-forecasting-understand-data/files/favorita-13-beverages-primary.csv -P ./sample-data

## get additional data
!wget https://oracle.github.io/learning-library/oci-library/oci-hol/oci-artificial-intelligence/forecasting-service/item-4-forecasting-understand-data/files/favorita-13-beverages-add.csv -P ./sample-data

--2022-03-20 06:28:30--  https://oracle.github.io/learning-library/oci-library/oci-hol/oci-artificial-intelligence/forecasting-service/item-4-forecasting-understand-data/files/favorita-13-beverages-primary.csv
Resolving oracle.github.io (oracle.github.io)... 185.199.111.153, 185.199.108.153, 185.199.109.153, ...
Connecting to oracle.github.io (oracle.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44537 (43K) [text/csv]
Saving to: ‘./sample-data/favorita-13-beverages-primary.csv.1’

100%[======================================>] 44,537      --.-K/s   in 0.1s    

2022-03-20 06:28:32 (342 KB/s) - ‘./sample-data/favorita-13-beverages-primary.csv.1’ saved [44537/44537]

--2022-03-20 06:28:32--  https://oracle.github.io/learning-library/oci-library/oci-hol/oci-artificial-intelligence/forecasting-service/item-4-forecasting-understand-data/files/favorita-13-beverages-add.csv
Resolving oracle.github.io (oracle.github.io)... 185.199.108.153, 

In [5]:
# Inline data preparation

import pandas as pd
import requests
import json
import ast
import matplotlib.pyplot as plt
import re
import os
import simplejson

In [6]:
# load data

df_primary = pd.read_csv('./sample-data/favorita-13-beverages-primary.csv')
df_add = pd.read_csv('./sample-data/favorita-13-beverages-add.csv')

# show data
print('*** primary data ***')
print(df_primary)

print('*** additional data ***')
print(df_add)

*** primary data ***
           item_id      date  sales
0     13_BEVERAGES  01/01/13      0
1     13_BEVERAGES  02/01/13    767
2     13_BEVERAGES  03/01/13    987
3     13_BEVERAGES  04/01/13    652
4     13_BEVERAGES  05/01/13   1095
...            ...       ...    ...
1665  13_BEVERAGES  28/07/17   1721
1666  13_BEVERAGES  29/07/17   2160
1667  13_BEVERAGES  30/07/17   1794
1668  13_BEVERAGES  31/07/17   2264
1669  13_BEVERAGES  01/08/17   1529

[1670 rows x 3 columns]
*** additional data ***
           item_id      date  onpromotion
0     13_BEVERAGES  01/01/13            0
1     13_BEVERAGES  02/01/13            0
2     13_BEVERAGES  03/01/13            0
3     13_BEVERAGES  04/01/13            0
4     13_BEVERAGES  05/01/13            0
...            ...       ...          ...
1679  13_BEVERAGES  11/08/17          218
1680  13_BEVERAGES  12/08/17          249
1681  13_BEVERAGES  13/08/17          226
1682  13_BEVERAGES  14/08/17          236
1683  13_BEVERAGES  15/08/17        

In [7]:
# modify data format of primary data
df_primary['date'] = pd.to_datetime(df_primary['date'], format='%d/%m/%y').apply(lambda date: str(date))

# modify data format of additional data
df_add['date'] = pd.to_datetime(df_add['date'], format='%d/%m/%y').apply(lambda date: str(date))

# show data
print('*** primary data ***')
print(df_primary)

print('*** additional data ***')
print(df_add)

*** primary data ***
           item_id                 date  sales
0     13_BEVERAGES  2013-01-01 00:00:00      0
1     13_BEVERAGES  2013-01-02 00:00:00    767
2     13_BEVERAGES  2013-01-03 00:00:00    987
3     13_BEVERAGES  2013-01-04 00:00:00    652
4     13_BEVERAGES  2013-01-05 00:00:00   1095
...            ...                  ...    ...
1665  13_BEVERAGES  2017-07-28 00:00:00   1721
1666  13_BEVERAGES  2017-07-29 00:00:00   2160
1667  13_BEVERAGES  2017-07-30 00:00:00   1794
1668  13_BEVERAGES  2017-07-31 00:00:00   2264
1669  13_BEVERAGES  2017-08-01 00:00:00   1529

[1670 rows x 3 columns]
*** additional data ***
           item_id                 date  onpromotion
0     13_BEVERAGES  2013-01-01 00:00:00            0
1     13_BEVERAGES  2013-01-02 00:00:00            0
2     13_BEVERAGES  2013-01-03 00:00:00            0
3     13_BEVERAGES  2013-01-04 00:00:00            0
4     13_BEVERAGES  2013-01-05 00:00:00            0
...            ...                  ...         

In [ ]:
# Setting variables to create forecast
primary_order = ['date', 'sales', 'item_id']
primary_load = df_primary[primary_order].values.transpose().tolist()
print(primary_load)

additional_order = ['date', 'onpromotion', 'item_id']
additional_load = df_add[additional_order].values.transpose().tolist()
print(additional_load)

In [9]:
# Set API authentication
import oci

auth = oci.auth.signers.InstancePrincipalsSecurityTokenSigner()

In [19]:
base_url = 'https://forecasting.aiservice.us-phoenix-1.oci.oraclecloud.com/20220101'
compartment_id = os.environ.get('C')
print(f'compartment_id:', compartment_id)

create_project_url = base_url + '/projects'

create_project_payload = json.dumps({
    'displayName': 'Forecast Demo',
    'compartmentId': compartment_id,
    'description': 'Forecasting service demo',
    'freeformTags': None,
    'definedTags': None,
    'systemTags': None,
})

create_project_header = {
    'Content-Type': 'application/json'
}

# Create project request
create_project_response = requests.request('POST', create_project_url, headers = create_project_header, data = create_project_payload ,auth = auth)

compartment_id: ocid1.compartment.oc1..aaaaaaaanjtbllhqxcg67dq7em3vto2mvsbc6pbgk4pw6cx37afzk3tngmoa


In [20]:
create_project_response_json = json.loads(create_project_response.text)
print(create_project_response_json)

{'id': 'ocid1.aiforecastproject.oc1.phx.amaaaaaassl65iqaf7ytsgxfiy4dj7u4qodfv76xth6v522vtezjnoktiopa', 'displayName': 'aiforecastproject20220320064320', 'compartmentId': 'ocid1.compartment.oc1..aaaaaaaanjtbllhqxcg67dq7em3vto2mvsbc6pbgk4pw6cx37afzk3tngmoa', 'description': 'Forecasting service demo', 'timeCreated': '2022-03-20T06:43:20.942Z', 'timeUpdated': '2022-03-20T06:43:20.942Z', 'lifecycleState': 'ACTIVE', 'freeformTags': {}, 'definedTags': {}, 'systemTags': {}}


In [21]:
project_id = create_project_response_json['id']
project_id

'ocid1.aiforecastproject.oc1.phx.amaaaaaassl65iqaf7ytsgxfiy4dj7u4qodfv76xth6v522vtezjnoktiopa'

In [24]:
date_col_primary = 'date'
date_col_add = 'date'
target_col = 'sales'
id_col_primary = 'item_id'
id_col_add = 'item_id'
data_frequency = 'DAY'
forecast_frequency = 'DAY'
forecast_horizon  = 7

In [30]:
%%time

create_forecast_url = base_url + '/forecasts'
create_forecast_url

create_forecast_payload = json.dumps({
  "compartmentId": compartment_id,
  "displayName": "Forecast Model",
  "description": "Training Forecast Model",
  "projectId": project_id,
  "forecastCreationDetails": {
    "targetVariables": [
      target_col
    ],
    "modelTrainingDetails": {
      "modelType": "UNIVARIATE",
      "models": [
        "SMA",
        "DMA",
        "HWSA",
        "HWSM",
        "SES",
        "DES",
        "SA",
        "SM",
        "UAM",
        "UHM",
        "ARIMA",
        "PROPHET"
      ]
    },
    "forecastHorizon": forecast_horizon,
    "confidenceInterval": "CI_5_95",
    "errorMeasure": "RMSE",
    "forecastTechnique": "ROCV",
    "forecastFrequency": forecast_frequency,
    "isForecastExplanationRequired": True,
    "dataSourceDetails": {
      "type": "INLINE",
      "dataSources": {
        "primaryDataSource": {
          "isDataGrouped": True,
          "columnData": primary_load,
          "columnSchema": [
            {
              "columnName": date_col_primary,
              "dataType": "DATE"
            },
            {
              "columnName": target_col,
              "dataType": "INT"
            },
            {
              "columnName": id_col_primary,
              "dataType": "STRING"
            },
          ],
          "tsColName": date_col_primary,
          "tsColFormat": "yyyy-MM-dd HH:mm:ss",
          "dataFrequency": data_frequency
        },
        "additionalDataSource": {
          "isDataGrouped": True,
          "columnData": additional_load,
          "columnSchema": [
            {
              "columnName": date_col_add,
              "dataType": "DATE"
            },
            {
              "columnName": "onpromotion",
              "dataType": "INT"
            },
            {
              "columnName": id_col_add,
              "dataType": "STRING"
            },
          ],
          "tsColName": date_col_add,
          "tsColFormat": "yyyy-MM-dd HH:mm:ss",
          "dataFrequency": data_frequency
        }
      }
    }
  }
})

create_forecast_headers = {
    'Content-Type': 'application/json'
}

create_forecast_response = requests.request('POST', create_forecast_url, headers = create_forecast_headers, data = create_forecast_payload, auth = auth)
create_forecast_response

CPU times: user 20.2 ms, sys: 0 ns, total: 20.2 ms
Wall time: 1.01 s


<Response [500]>

In [26]:
create_forecast_response_json = json.loads(create_forecast_response)
create_forecast_response_json

TypeError: the JSON object must be str, bytes or bytearray, not 'Response'